# Text Analytics

自然言語処理（NLP）は、書き言葉や話し言葉を扱う人工知能（AI）の一分野です。NLPを使用して、テキストや音声から意味的な意味を抽出したり、自然言語で意味のある応答を定式化したりするソリューションを構築することができます。

Microsoft Azureの*Cognitive Services*には、*Text Analytics*サービスが含まれており、テキスト内のキーフレーズの識別や、センチメントに基づいたテキストの分類など、すぐに使えるNLP機能を提供しています。

<p style='text-align:center'><img src='./images/NLP.jpg' alt='ノートを読むロボット'/></p>

例えば、架空の組織*Margie's Travel*が、顧客にホテル宿泊のレビュー投稿を奨励しているとします。テキスト分析サービスを使用して、キーフレーズを抽出してレビューを要約したり、どのレビューが肯定的でどのレビューが否定的かを判断したり、レビューのテキストを分析して、場所や人物などの既知のエンティティについて言及しているかどうかを確認したりすることができます。

## レビューのドキュメントを見る

まずは、お客様が残してくださったホテルのレビューを見てみましょう。

レビューはテキストファイルです。レビューを見るには、下のセルの緑色の<span style="color:green">&#9655;</span>ボタン（セルの左上）をクリックしてください。

In [ ]:
import os

# data/reviewsフォルダ内のレビューを読む
reviews_folder = os.path.join('data', 'text', 'reviews')

# id (ファイル名) と text (内容) のプロパティを持つレビューのコレクションを作成するreviews = []
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # レビューのテキストを表示します
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Cognitive Servicesリソースの作成

これらのレビューのテキストを分析するには、**Text Analytics**コグニティブサービスを使用できます。これを使用するには、Azureサブスクリプションで**Text Analytics**または**Cognitive Services**リソースのいずれかをプロビジョニングする必要があります(これが唯一のサービスであるか、またはその使用状況を個別に追跡したい場合は、Text Analyticsリソースを使用してください。それ以外の場合は、コグニティブサービスリソースを使用してText Analyticsサービスを他のコグニティブサービスと組み合わせることができます。)

まだ持っていない場合は、次の手順を使用して、Azureサブスクリプションで**Cognitive Services**リソースを作成します。

1. 別のブラウザタブで、https://portal.azure.com の Azure ポータルを開き、Microsoft アカウントでサインインします。
2. **[&#65291;リソースの作成]**ボタンをクリックして、*Cognitive Services*を検索し、次の設定で**Cognitive Services**リソースを作成します。:
    - **Name**:  *一意の名前を入力してください*
    - **Subscription**: *あなたのAzureサブスクリプション*
    - **Location**: *利用可能な場所*
    - **Pricing tier**: S0
    - **Resource group**: *固有の名前を持つリソースグループ*を作成
3. デプロイが完了するのを待ちます。次に、コグニティブサービスリソースに移動し、**クイックスタート**ページで、キーとエンドポイントに注意してください。クライアント アプリケーションからコグニティブ サービス リソースに接続するには、これらが必要です。

## Cognitive Servicesリソースのキーとエンドポイントを取得する

認知サービス リソースを使用するには、クライアント アプリケーションにそのエンドポイントと認証キーが必要です。

1. Azure ポータルで、コグニティブサービスリソースの **クイックスタート** ページで、リソースの **Key1** をコピーして、**YOUR_COG_KEY** を置き換えて、以下のコードに貼り付けます。
2. 2. リソースの **endpoint** をコピーして、以下のコードに貼り付けて、 **YOUR_COG_ENDPOINT** を置き換えます。
3. 3. 以下のセルを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

これでCognitive Servicesのセットアップが完了しました。

そしてPythonから本演習を実行するには、まずAzureの関連パッケージをインストールする必要があります。

In [ ]:
! pip install azure-cognitiveservices-language-textanalytics

## 言語の検出
まずは、これらのレビューがどのような言葉で書かれているのかを見極めることから始めましょう。

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## キーフレーズを抽出する

今、あなたは、主要な話のポイントのいくつかの兆候を与えるキーフレーズを識別するために、カスタマーレビューのテキストを分析することができます。

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

キーフレーズは、各レビューの中で最も重要なトークポイントを理解するのに役立ちます。例えば、"親切なスタッフ "や "サービスが悪い "というフレーズが含まれているレビューは、レビュアーの主な関心事のいくつかを示すことができます。

## センチメントを決定する

センチメントスコア*に基づいて、レビューを*肯定的*または*否定的*に分類すると便利かもしれません。繰り返しになりますが、これを行うにはテキストアナリティクスサービスを使用することができます。

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## 既知のエンティティを抽出する

*エンティティ*とは、一般的に理解されているタイプのアイテムを参照している文章の中で言及される可能性のあるものです。例えば、場所、人、日付などです。例えば、レビューで言及されているdateaや場所に興味があるとします - それらを見つけるには以下のコードを使用できます。

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only get location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

一部のエンティティは、ウィキペディアに関連するページを持っていることが十分に知られていることに注意してください。

## Learn More

Text Analyticsサービスの詳細については、[Text Analyticsサービスのドキュメント](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)を参照してください。